In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")

import pykep as pk
import numpy as np
import matplotlib.pyplot as plt
import math
import os

starting_t = pk.epoch_from_string('2022-01-01 00:00:00.000')
lower_cutoff_in_km = 6371 + 200 # Earth radius + ...
higher_cutoff_in_km = 6371 + 2000

# Initialize / reset planet_list and other

In [ ]:
planet_list = []
shell_list = []
xparams_list = []
print("Planet list is now empty")

# Generate partial constellations by specifying shell = ( altitude, inclination, nPlanes, nSats )

In [ ]:
# creates a temporary shell that has to be appended to planet_list in cell "Append to planet_list and other"
# to be stored and not get overwritten (this also concerns additional shell data shell_list, xparams_list)

#---input-----------------------------------------------------------------------------------------------------------
altitude = 370
inclination = 55.0
nPlanes = 10
nSats = 12
#-------------------------------------------------------------------------------------------------------------------
#---extra-params----------------------------------------------------------------------------------------------------

#offsetM = offset for M added after each plane (relative phasing)
# walker constellation: offsetM = F * 360 / (nPlanes * nSats) ; F element {0, ... , nPlanes - 1}
offsetM = 0    # default 0

#startingM = offset for M that is not accumulating
# overlapping shells (same altitude, same inclination):
# (360 / G) / 2 ; G = smallest common multiple of the overlapping nPlanes
startingM = 0  # default 0 

#startingW = offset for W that is not accumulating
startingW = 0  # default 0

#minimumW, maximumW: orbital planes are distributed evenly within range [minimumW,maximumW)
minimumW = 0       # default 0
maximumW = 360     # default 360
#-------------------------------------------------------------------------------------------------------------------

minW = minimumW + startingW
maxW = maximumW + startingW

a = altitude*1000 + 6371000            # in [m], earth radius included
e = 0
i = inclination * pk.DEG2RAD
W = pk.DEG2RAD*minW
w = 0
M = pk.DEG2RAD * startingM

plane_count=0

when = starting_t
mu_central_body = pk.MU_EARTH
mu_self = 1
radius = 1
safe_radius = 1

pStep = pk.DEG2RAD*(maxW-minW)/ nPlanes  # W goes from minimumW to maximumW (default: 0° to 360°)
sStep = 2*math.pi/ nSats                    # M goes from 0° to 360°
sExtraStep = pk.DEG2RAD*offsetM

planet_list_tmp = []
for x in range(nPlanes):
    for y in range(nSats):
        planet_list_tmp.append(pk.planet.keplerian(when,[a,e,i,W,w,M], mu_central_body, mu_self,radius, safe_radius ,"sat"))
        M = M + sStep
    plane_count = plane_count + 1
    W = W + pStep
    M = startingM + plane_count*sExtraStep     #equals 0 + count*0 = 0 in the usual case
    
shell = (altitude,inclination,nPlanes,nSats)
xparams = (offsetM,startingM,startingW,minimumW,maximumW)

print("Added " + str(len(planet_list_tmp)) + " planets")

## Append to planet_list and other

In [ ]:
# stores and appends temporary shell in planet_list (including lists with additional data)
planet_list = planet_list + planet_list_tmp
shell_list.append(shell)
xparams_list.append(xparams)
print("Added " + str(len(planet_list_tmp)) + " to planet_list now totalling " + str(len(planet_list)) + " planets.")

## Plot some examples

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d')
for i in range (nPlanes*nSats):
    pk.orbit_plots.plot_planet(planet_list_tmp[i],axes=ax,s=20)

## Propagate all objects to t and discard too low and high ones

In [ ]:
objects = []
count_too_low = 0
count_too_high = 0
for planet in planet_list:
    try:
        pos,v = planet.eph(starting_t)
        
        # convert to km and numpy
        pos = np.asarray(pos) / 1000.0 
        v = np.asarray(v) / 1000.0
        altitude = np.linalg.norm(pos)
        if altitude < lower_cutoff_in_km:
            count_too_low += 1
            continue
        if altitude > higher_cutoff_in_km:
            count_too_high += 1
            continue
        
        objects.append((pos,v))
    except RuntimeError as e:
        print(e, " propagating ",planet.name)
        
print("Successfully propagated ",len(objects)," objects.")
print(count_too_low," had a too small altitude")
print(count_too_high," had a too high altitude")

shells = ""
for i in range(len(shell_list)):
    for j in range(4):
        shells += str(shell_list[i][j]) + " "
    shells += "\n"
    
extraParams = ""
for i in range(len(xparams_list)):
    for j in range(5):
        extraParams += str(xparams_list[i][j]) + " "
    extraParams += "\n"

## Plot and store results

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d');

positions = np.array([pos for pos,_ in objects])
velocities = np.array([v for _,v in objects])
ax.scatter(positions[:,0],positions[:,1],positions[:,2],".",alpha=0.25)

In [ ]:
# creates directory named as the constellation_name value within the data folder and stores output there

constellation_name = "Constellation"
os.mkdir("../../data/" + constellation_name);

np.savetxt("../../data/" + constellation_name + "/pos_"+constellation_name+".csv",positions,delimiter=",")
np.savetxt("../../data/" + constellation_name + "/v_"+constellation_name+".csv",velocities,delimiter=",")

f1 = open("../../data/" + constellation_name + "/shells_"+constellation_name+".txt","w")
f1.write(shells)
f1.close()

f2 = open("../../data/" + constellation_name + "/xparams_"+constellation_name+".txt","w")
f2.write(extraParams)
f2.close()